### corrupt_json_multiline_ingestion

In [1]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown('# <span style="color:red">'+string+'</span>'))

if ('sc' in locals() or 'sc' in globals()):
    printmd('<<<<<!!!!! It seems that you are running in a IBM Watson Studio Apache Spark Notebook. Please run it in an IBM Watson Studio Default Runtime (without Apache Spark) !!!!!>>>>>')

!pip install pyspark==2.4.5

try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')
    
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

print("Apache Spark session created.")

Apache Spark session created.


In [2]:
"""
   Failing multiline ingestion JSON ingestion in a dataframe.
   This example illustrates what happens when you forget the multiline
   option and try to ingest a multiline JSON file.
   Output is:
   <pre>
   +--------------------+
   |     _corrupt_record|
   +--------------------+
   |                 [ {|
   |       "tag" : "A1",|
   |  "geopoliticalar...|
   +--------------------+
   only showing top 3 rows
   root
   |-- _corrupt_record: string (nullable = true)
  * </pre>
   The data comes from The Bureau of Consular Affairs of the US Department
   of State. You can access their open data portal at
   https://cadatacatalog.state.gov/.
"""
from pyspark.sql import SparkSession
import os

current_dir = os.path.dirname("")
relative_path = "countrytravelinfo.json"
absolute_file_path = os.path.join(current_dir, relative_path)

# Creates a session on a local master
spark = SparkSession.builder.appName("Multiline JSON to Dataframe, without multiline option") \
    .master("local[*]").getOrCreate()

# Reads a JSON, called countrytravelinfo.json, stores it in a
# dataframe,
# without specifying the multiline option
df = spark.read.format("json") \
          .load(absolute_file_path)

# Shows at most 3 rows from the dataframe
df.show(3)
df.printSchema()

spark.stop()

AnalysisException: 'Since Spark 2.3, the queries from raw JSON/CSV files are disallowed when the\nreferenced columns only include the internal corrupt record column\n(named _corrupt_record by default). For example:\nspark.read.schema(schema).json(file).filter($"_corrupt_record".isNotNull).count()\nand spark.read.schema(schema).json(file).select("_corrupt_record").show().\nInstead, you can cache or save the parsed results and then send the same query.\nFor example, val df = spark.read.schema(schema).json(file).cache() and then\ndf.filter($"_corrupt_record".isNotNull).count().;'